In [11]:
%pip install redis



Note: you may need to restart the kernel to use updated packages.


In [12]:

import redis

r = redis.Redis(host='localhost', port=6379, db=0)

In [13]:
from counter import get_last_counter

bank_id = f"bank_{get_last_counter()}"
bank_id

'bank_44'

In [14]:
r.keys()

[b'faiss_index:v1::bank_40',
 b'faiss_index:v1::bank_39',
 b'memory_banks:v1::bank_41',
 b'faiss_index:v1::bank_42',
 b'faiss_index:v1::bank_44',
 b'memory_banks:v1::bank_42',
 b'memory_banks:v1::bank_44',
 b'memory_banks:v1::bank_43',
 b'faiss_index:v1::bank_41',
 b'faiss_index:v1::bank_38',
 b'memory_banks:v1::bank_38',
 b'memory_banks:v1::bank_40',
 b'memory_banks:v1::bank_39']

In [15]:
r.get(f"memory_banks:v1::{bank_id}")

b'{"identifier":"bank_44","provider_resource_id":"bank_44","provider_id":"faiss","type":"memory_bank","memory_bank_type":"vector","embedding_model":"all-MiniLM-L6-v2","chunk_size_in_tokens":512,"overlap_size_in_tokens":null}'

In [16]:
r.info()

{'redis_version': '7.4.1',
 'redis_git_sha1': 0,
 'redis_git_dirty': 0,
 'redis_build_id': '7fd3520e9c14a41b',
 'redis_mode': 'standalone',
 'os': 'Linux 6.8.0-49-generic x86_64',
 'arch_bits': 64,
 'monotonic_clock': 'POSIX clock_gettime',
 'multiplexing_api': 'epoll',
 'atomicvar_api': 'c11-builtin',
 'gcc_version': '11.4.0',
 'process_id': 4389,
 'process_supervised': 'systemd',
 'run_id': 'fd477aea774b4109d9b19beeb8eb6d46298a1c58',
 'tcp_port': 6379,
 'server_time_usec': 1733655022691808,
 'uptime_in_seconds': 3356,
 'uptime_in_days': 0,
 'hz': 10,
 'configured_hz': 10,
 'lru_clock': 5601774,
 'executable': '/usr/bin/redis-server',
 'config_file': '/etc/redis/redis.conf',
 'io_threads_active': 0,
 'listener0': {'name': 'tcp', 'bind': '-::1', 'port': 6379},
 'connected_clients': 2,
 'cluster_connections': 0,
 'maxclients': 10000,
 'client_recent_max_input_buffer': 8,
 'client_recent_max_output_buffer': 0,
 'blocked_clients': 0,
 'tracking_clients': 0,
 'pubsub_clients': 0,
 'watchin

In [17]:
from json import loads
from IPython.display import display

index = loads(r.get(f"faiss_index:v1::{bank_id}"))
# Write index to JSON file
import json
with open('faiss_index.json', 'w') as f:
    json.dump(index, f, indent=2)
display(index.keys())
display(len(index['id_by_index']))

KeyboardInterrupt: 

In [7]:
import os
from llama_stack_client import LlamaStackClient

# Test run
client = LlamaStackClient(
    base_url="http://localhost:5001",
)

response = client.memory.query(
    bank_id=bank_id,
    query=["Hello World"],
    params={"max_chunks": 10},
)

display(response)

for chunk, score in zip(response.chunks, response.scores):
    print(chunk)
    print(score)


QueryDocumentsResponse(chunks=[Chunk(content=' combat to the bitter -- URFFF!\n          Toothless rests his head on Hiccup -- SQUEEZING THE AIR out\n          of him. Toothless then suddenly starts grooming Hiccup,\n          lapping at his face and covering him in slobber.\n\n                         \n\n<b>                          HICCUP (CONT\'D)\n</b>           Gaaaaagh!\n\n                         \n          Hiccup fights to get free, finally springing to his feet and\n          shaking off the viscous saliva.\n           HTTYD 2 (D. DeBlois) - FINAL DRAFT - 05/05/14 10.\n\n                         \n\n                         \n\n<b>                          HICCUP (CONT\'D)\n</b>           You KNOW that doesn\'t wash out.\n\n                         \n          Toothless LAUGHS, pleased with himself.\n\n                         \n          Shifting his attention back to the new land stretched out\n          before them, Hiccup reaches beneath his chest plate and\n          pr

Chunk(content=' combat to the bitter -- URFFF!\n          Toothless rests his head on Hiccup -- SQUEEZING THE AIR out\n          of him. Toothless then suddenly starts grooming Hiccup,\n          lapping at his face and covering him in slobber.\n\n                         \n\n<b>                          HICCUP (CONT\'D)\n</b>           Gaaaaagh!\n\n                         \n          Hiccup fights to get free, finally springing to his feet and\n          shaking off the viscous saliva.\n           HTTYD 2 (D. DeBlois) - FINAL DRAFT - 05/05/14 10.\n\n                         \n\n                         \n\n<b>                          HICCUP (CONT\'D)\n</b>           You KNOW that doesn\'t wash out.\n\n                         \n          Toothless LAUGHS, pleased with himself.\n\n                         \n          Shifting his attention back to the new land stretched out\n          before them, Hiccup reaches beneath his chest plate and\n          produces a leather-bound book.\n\

In [8]:
import pandas as pd

datasets = {
    "narrativeqa": "data/narrativeqa.parquet",
}

df = pd.read_parquet(datasets["narrativeqa"])
df['context_length'] = df['context'].apply(lambda x: len(x))
df.describe()

,context_length
count,50.000000
mean,217480.920000
std,58733.594823
min,110830.000000
25%,186267.000000
50%,215859.500000
75%,240608.500000
max,417011.000000


In [9]:
doc_ids = []
questions = []
for _, row in df.iterrows():
    for question in row["questions"]:
        questions.append(question)
        doc_ids.append(_)

print(len(questions))
print(len(doc_ids))
display(questions[:5])
doc_ids[:40]

1490
1490


['Who planned the robbery that was being investigated when Domino Harvey was arrested?',
 'Which career was Domino in before she became a bounty hunter?',
 "What did Lateesha plan to use to pay for her granddaughter's medical procedure?",
 'How did Claremont know there would be $10 Million in the armored car?',
 'Who were the FBI initially led to believe would commit the robbery?']

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [10]:
from tqdm import tqdm

top_k = 10

results = []

for i in tqdm(range(0, len(questions))):
    response = client.memory.query(
        bank_id=bank_id,
        query=[questions[i]],
        params={"max_chunks": top_k},
    )
    res = [int(chunk.document_id) for chunk in response.chunks]
    results.append(res)

results

  1%|          | 18/1490 [00:00<00:16, 87.63it/s]

100%|██████████| 1490/1490 [08:15<00:00,  3.01it/s]


[[0, 0, 0, 0, 0, 45, 0, 0, 45, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 28, 0, 0, 17, 0, 42, 49, 0, 49],
 [0, 0, 0, 0, 11, 35, 10, 0, 21, 0],
 [45, 0, 44, 28, 45, 10, 38, 10, 0, 28],
 [0, 0, 0, 30, 15, 23, 28, 2, 25, 39],
 [0, 0, 0, 10, 0, 10, 10, 10, 35, 10],
 [34, 18, 18, 34, 38, 34, 34, 34, 34, 34],
 [10, 10, 10, 0, 45, 0, 44, 38, 10, 10],
 [0, 0, 0, 0, 0, 35, 5, 0, 0, 35],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 16],
 [2, 2, 9, 2, 42, 0, 42, 28, 38, 0],
 [0, 0, 29, 10, 0, 36, 0, 10, 0, 15],
 [0, 0, 0, 0, 10, 0, 10, 10, 0, 36],
 [0, 0, 0, 0, 45, 44, 0, 19, 0, 0],
 [0, 0, 29, 29, 0, 10, 35, 29, 7, 35],
 [10, 22, 10, 10, 10, 17, 34, 10, 4, 11],
 [0, 0, 11, 11, 0, 11, 11, 11, 11, 11],
 [34, 0, 34, 18, 0, 0, 18, 34, 34, 34],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [32, 32, 49, 30, 15, 49, 49, 2, 49, 0],
 [0, 21, 0, 44, 28, 21, 0, 0, 10, 10],
 [0, 28, 37, 0, 44, 15, 45, 38, 35, 2],
 [16, 42, 22, 31, 30, 16, 0, 23, 16, 30],
 [0, 0, 10, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 

In [18]:
import numpy as np

targets = np.array(doc_ids)
display(targets.shape)
targets = np.expand_dims(targets, axis=1)
display(targets.shape)
targets = targets == res
display(targets.shape)
targets = targets.astype(float)
display(targets)

(1490,)

(1490, 1)

(1490, 10)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [27]:
top_k = 3

res = np.array(results)
res = res[:, :top_k]
targets = targets[:, :top_k]
display(targets.shape)
display(res.shape)

(1490, 3)

(1490, 3)

In [28]:
temp = np.sum(targets, axis=1)
temp = np.divide(temp, top_k)
precision = np.sum(temp) / len(temp)
precision

np.float64(0.019686800894854584)

In [29]:
temp = np.logical_or.reduce(targets, axis=1)
temp = temp.astype(float)
recall = np.sum(temp) / len(temp)
recall

np.float64(0.05906040268456376)

In [30]:
ranks = np.tile(np.arange(1, top_k + 1), (len(questions), 1)).astype(float)
temp = np.copy(targets)
for i in range(len(temp)):
    first_true = True
    for j in range(len(temp[i])):
        if temp[i,j]:
            if first_true:
                first_true = False
            else:
                temp[i,j] = False
reciprocal_ranks = temp / ranks
reciprocal_ranks = np.sum(reciprocal_ranks) / len(temp)
reciprocal_ranks = np.mean(reciprocal_ranks)
reciprocal_ranks

np.float64(0.036241610738255034)